<a href="https://colab.research.google.com/github/vpste1/mlscrapbook/blob/master/BaselineMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 12.8MB/s 
     |████████████████████████████████| 1.1MB 58.0MB/s 
     |████████████████████████████████| 51kB 9.7MB/s 
     |████████████████████████████████| 194kB 57.3MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 
Mounted at /content/gdrive


In [ ]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

Load the training sets

In [ ]:
train_3_tens = torch.stack([tensor(Image.open(o)) 
                            for o in (path/'train'/'3').ls()])
train_3_tens = train_3_tens.float()/255
train_7_tens = torch.stack([tensor(Image.open(o)) 
                            for o in (path/'train'/'7').ls()])
train_7_tens = train_7_tens.float()/255
train_3_tens.shape,train_7_tens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

Load the validation sets

In [ ]:
valid_3_tens = torch.stack([tensor(Image.open(o)) 
                            for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o)) 
                            for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
valid_3_tens.shape,valid_7_tens.shape

(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

Generate the means of the training set

In [ ]:
mean3 = train_3_tens.mean(0)
mean7 = train_7_tens.mean(0)

mean3.shape

torch.Size([28, 28])

Define a function to return the mean absolute error (L1 norm).

Specifying the tuple `(-1, -2)` is redundant when inputting two rank-2 tensors. (e.g. `.mean()` would end up with the same output.

It is however useful when making use of broadcasting; e.g. if you were to send in a rank-3 as `a` and a rank-2 as `b`; we specified the "last two" axes with `(-1, -2)`; which will output a rank-1 (list) of distances (rather than a single mnist difference). 

*Broadcasting behind the scenes is PyTorch automatically "expanding" the tensor with the smaller rank to have the same **size** as the one with the larger rank. (it doesn't actually allocate memory to this expansion however)*

In [ ]:
def mnist_distance(a,b): return (a-b).abs().mean((-1,-2))

In [ ]:
mnist_distance(train_3_tens, mean3)

tensor([0.1280, 0.1496, 0.1128,  ..., 0.1381, 0.1183, 0.1183])

We can make use of this function to define a simple "is-3" qualifier:

In [ ]:
def is_3(x): return mnist_distance(x,mean3) < mnist_distance(x,mean7)

In [ ]:
is_3(valid_3_tens)

tensor([True, True, True,  ..., True, True, True])

A list of booleans needs to be converted to floats in order to calculate the accuracy. The accuracy of the 7s will be when they are "not 3"; and the overall accuracy across both sets is the average of these two.

In [ ]:
accuracy_3s =      is_3(valid_3_tens).float() .mean()
accuracy_7s = (1 - is_3(valid_7_tens).float()).mean()

accuracy_3s,accuracy_7s,(accuracy_3s+accuracy_7s)/2

(tensor(0.9168), tensor(0.9854), tensor(0.9511))

We can say our "3-detector" is around 95% accurate despite being a primitive solution. Establishing a baseline can help compare future models as it can be hard to tell if "fancy" models are actually any good.